# Support Vector Machine: Predicting CHD

Here is an intersting problem of understanding what factors contribute to CHD and can CHD be predicted by building an analytical model.

The next two sections will introduce some basics of CHD, where the dataset is derived from and what are the attributes available in the dataset.

### What is coronary heart disease?


[Coronary heart disease (CHD)](https://en.wikipedia.org/wiki/Coronary_artery_disease)  is when your coronary arteries (the arteries that supply your heart muscle with oxygen-rich blood) become narrowed by a gradual build-up of fatty material within their walls. These arteries can become narrowed through build-up of plaque, which is made up of cholesterol and other substances. Narrowed arteries can cause symptoms, such as chest pain (angina), shortness of breath, and fatigue.


### Dataset Description

Data is avaialable at: http://statweb.stanford.edu/~tibs/ElemStatLearn/datasets/
And header informtion is available at: http://statweb.stanford.edu/~tibs/ElemStatLearn/datasets/SAheart.info.txt

A retrospective sample of **males in a heart-disease high-risk region of the Western Cape, South Africa**. There are roughly two controls per case of CHD. Many of the CHD positive men have undergone blood pressure reduction treatment and other programs to reduce their risk factors after their CHD event. In some cases the measurements were made after these treatments. These data are taken from a larger dataset, described in Rousseauw et al, 1983, South African Medical Journal. 

### Import and load the dataset

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
saheart_ds = pd.read_csv( "https://drive.google.com/uc?export=download&id=1yRyZMfBQ8anG10GDFsLf15GRYPQqN12b" )

In [ ]:
saheart_ds.head()

In [ ]:
saheart_ds.columns

In [ ]:
saheart_ds.info()

In [ ]:
saheart_ds.chd.value_counts(normalize=True)

The class label int the column **chd** indicates if the person has a coronary heart disease: negative (0) or positive (1). 

Attributes description: 
- **sbp**:          systolic blood pressure 
- **tobacco**:      cumulative tobacco (kg) 
- **ldl**:          low densiity lipoprotein cholesterol 
- **adiposity**:    the size of the hips compared to the person's height 
- **famhist**:      family history of heart disease (Present, Absent) 
- **typea**:        type-A behavior 
- **obesity**:      BMI index
- **alcohol**:      current alcohol consumption 
- **age**:          age at onset

### Predict CHD based

In [ ]:
feature_cols = ['age', 'ldl']

In [ ]:
plt.figure(figsize=(8, 6))
sn.scatterplot(data=saheart_ds,
          x = feature_cols[0],
          y = feature_cols[1],
          hue = 'chd');

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:

saheart_train_X, saheart_test_X,  \
saheart_train_Y, saheart_test_Y = train_test_split( saheart_ds[feature_cols],
                                                    saheart_ds['chd'],
                                                    test_size = 0.3,
                                                    random_state = 42 ) 

In [ ]:
saheart_train_X.shape

In [ ]:
from sklearn.utils.class_weight import compute_class_weight

In [ ]:
class_weights = compute_class_weight(class_weight = "balanced", 
                                     classes = np.unique(saheart_train_Y), 
                                     y = saheart_train_Y)

In [ ]:
class_weights

In [ ]:
class_weight={1:1.45, 0:0.761}

### Linear SVC

In [ ]:
from sklearn.svm import SVC

svc = SVC(kernel = 'linear', class_weight=class_weight)

In [ ]:
svc.fit( saheart_train_X, saheart_train_Y )

In [ ]:
svc_pred_y = svc.predict(saheart_test_X)

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

In [ ]:
cm = confusion_matrix(saheart_test_Y, svc_pred_y, labels=[1,0])

In [ ]:
ConfusionMatrixDisplay(cm, display_labels = ['CHD', 'No CHD']).plot();

In [ ]:
def plot_svc_decision_function(model, ax=None, plot_support=True):
    """Plot the decision function for a 2D SVC"""
    if ax is None:
        ax = plt.gca()
    xlim = ax.get_xlim()
    ylim = ax.get_ylim()
    
    # create grid to evaluate model
    x = np.linspace(xlim[0], xlim[1], 30)
    y = np.linspace(ylim[0], ylim[1], 30)
    Y, X = np.meshgrid(y, x)
    xy = np.vstack([X.ravel(), Y.ravel()]).T
    P = model.decision_function(xy).reshape(X.shape)
    
    # plot decision boundary and margins
    ax.contour(X, Y, P, colors='k',
               levels=[-1, 0, 1], alpha=0.5,
               linestyles=['--', '-', '--'])
    
    # plot support vectors
    if plot_support:
        ax.scatter(model.support_vectors_[:, 0],
                   model.support_vectors_[:, 1],
                   s=300, linewidth=1, facecolors='none');
    ax.set_xlim(xlim)
    ax.set_ylim(ylim)

In [ ]:
plt.figure(figsize=(8, 6))
plt.scatter(saheart_train_X[feature_cols[0]],
            saheart_train_X[feature_cols[1]], 
            c=saheart_train_Y, 
            s=50, 
            cmap='coolwarm')
plot_svc_decision_function(svc);

### Polynomial SVC

In [ ]:
ploy_svc = SVC(kernel = 'poly', degree = 2, class_weight=class_weight)

ploy_svc.fit( saheart_train_X, saheart_train_Y )

svc_pred_y = ploy_svc.predict(saheart_test_X)
cm = confusion_matrix(saheart_test_Y, svc_pred_y, labels=[1,0])
ConfusionMatrixDisplay(cm, display_labels = ['CHD', 'No CHD']).plot();

In [ ]:
plt.figure(figsize=(8, 6))
plt.scatter(saheart_train_X[feature_cols[0]],
            saheart_train_X[feature_cols[1]], 
            c=saheart_train_Y, 
            s=50, 
            cmap='coolwarm')
plot_svc_decision_function(ploy_svc);

In [ ]:
rbf_svc = SVC(kernel = 'rbf', class_weight=class_weight)

rbf_svc.fit( saheart_train_X, saheart_train_Y )

svc_pred_y = rbf_svc.predict(saheart_test_X)
cm = confusion_matrix(saheart_test_Y, svc_pred_y, labels=[1,0])
ConfusionMatrixDisplay(cm, display_labels = ['CHD', 'No CHD']).plot();

In [ ]:
plt.figure(figsize=(8, 6))
plt.scatter(saheart_train_X[feature_cols[0]],
            saheart_train_X[feature_cols[1]], 
            c=saheart_train_Y, 
            s=50, 
            cmap='coolwarm')
plot_svc_decision_function(rbf_svc);

### With All the features

In [ ]:
saheart_model_df = pd.get_dummies( saheart_ds, 
                                   columns = ['famhist'],
                                   drop_first = True)

In [ ]:
saheart_model_df.head( 10 )

In [ ]:
saheart_model_df = saheart_model_df.drop( "row.names", axis = 1 )

In [ ]:
saheart_model_df.columns

## Splitting Dataset into Train and Test

In [ ]:
feature_cols = ['sbp', 'tobacco', 'ldl', 
                'adiposity', 'typea', 
                'obesity', 'alcohol', 
                'age', 'famhist_Present' ]

saheart_train_X, saheart_test_X,  \
saheart_train_Y, saheart_test_Y = train_test_split( saheart_model_df[feature_cols],
                                                    saheart_model_df['chd'],
                                                    test_size = 0.3,
                                                    random_state = 42 ) 

In [ ]:
len( saheart_train_X )

In [ ]:
len( saheart_test_X )

## Building SVC Model

In [ ]:
from sklearn.svm import SVC

rbf_svc = SVC(kernel = 'rbf', class_weight=class_weight, probability=True)

In [ ]:
rbf_svc.fit( saheart_train_X, saheart_train_Y )

### Predicting in test set using the model

In [ ]:
saheart_test_results = pd.DataFrame( { 'actual':  saheart_test_Y, 
                                       'predicted': rbf_svc.predict( saheart_test_X ) } )

In [ ]:
saheart_test_results = saheart_test_results.reset_index()
saheart_test_results.head()

## Model Evaluation

### Confusion Matrix

In [ ]:
cm = confusion_matrix(saheart_test_results.actual, saheart_test_results.predicted, labels=[1,0])
ConfusionMatrixDisplay(cm, display_labels = ['CHD', 'No CHD']).plot();

In [ ]:
from sklearn.metrics import RocCurveDisplay

In [ ]:
RocCurveDisplay.from_estimator(rbf_svc, saheart_test_X, saheart_test_Y);